In [1]:
import torch
_ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])

/var/folders/3t/hpl6nvhn3zzfqzdm_l66mdyc0000gn/T/ipykernel_22170/97062541.py:2: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/c10/core/TensorImpl.h:1791.)
  _ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])


In [17]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [18]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

In [19]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [44]:
img_t.shape, weights.shape

(torch.Size([3, 5, 5]), torch.Size([3]))

In [45]:
img_t * weights

RuntimeError: The size of tensor a (5) must match the size of tensor b (3) at non-singleton dimension 2

In [22]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1)
unsqueezed_weights.shape

torch.Size([3, 1, 1])

In [23]:
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]))

In [24]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [26]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [27]:
img_named =  img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [28]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [29]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [33]:
img_named[..., :3].shape

torch.Size([3, 5, 3])

In [38]:
img_named[..., :3] * weights_aligned

tensor([[[-2.2077e-01,  1.6918e-02,  1.9756e-01],
         [-1.3846e-01, -2.3480e-01, -4.6207e-01],
         [-1.5649e-03,  3.0898e-01,  1.6445e-02],
         [-6.6749e-02,  2.6759e-01,  4.8013e-03],
         [ 3.0129e-01,  1.1571e-01, -3.6799e-01]],

        [[-2.3219e-01,  1.2695e+00, -5.9444e-01],
         [-4.1607e-01, -8.8187e-01, -4.7309e-01],
         [-6.9385e-01,  1.8836e+00, -2.0557e-01],
         [-4.7585e-01,  4.3205e-01,  1.6532e-01],
         [-4.8894e-01, -3.5375e-01,  4.7881e-02]],

        [[-7.8375e-03, -1.1237e-02,  3.9116e-02],
         [-1.0323e-01, -7.6315e-02,  2.3942e-02],
         [ 1.6431e-01,  2.9945e-02, -5.6564e-02],
         [-1.2770e-01,  1.5648e-01, -4.8505e-02],
         [ 7.6615e-02, -8.8173e-02, -4.0922e-02]]],
       names=('channels', 'rows', 'columns'))

In [11]:
try:
    gray_named = (img_named[..., :3] * weights_named).sum('channels')
except Exception as e:
    print(e)

Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.


In [39]:
try:
    gray_named = (img_named[..., :3] * weights_aligned).sum('channels')
except Exception as e:
    print(e)

In [41]:
gray_named.shape

torch.Size([5, 3])

In [12]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))